## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F399B21130>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F399C44230>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish, it's nice to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you working on these days? I'm always interested to hear about the cutting edge of AI development.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 22, 'total_tokens': 73, 'completion_time': 0.092727273, 'prompt_time': 0.002133055, 'queue_time': 0.248191964, 'total_time': 0.094860328}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--699d12a1-4bec-4b6d-b3ff-4984d221dd8d-0', usage_metadata={'input_tokens': 22, 'output_tokens': 51, 'total_tokens': 73})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Krish, and you are a Chief AI Engineer. \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 97, 'total_tokens': 134, 'completion_time': 0.067272727, 'prompt_time': 0.00432652, 'queue_time': 0.24388235, 'total_time': 0.071599247}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--386639e2-5742-4e95-8f90-e8ff8a3157c3-0', usage_metadata={'input_tokens': 97, 'output_tokens': 37, 'total_tokens': 134})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}
config

{'configurable': {'session_id': 'chat1'}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [8]:
response.content

"Hi Krish! It's nice to meet you. \n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you working on these days?  I'm always eager to learn more about the cutting edge of AI development.\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish. 😊 \n\nI remember that from our introduction!  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 90, 'total_tokens': 119, 'completion_time': 0.052727273, 'prompt_time': 0.004075531, 'queue_time': 0.242651122, 'total_time': 0.056802804}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c3bedb65-d16a-404f-b915-e8e6de272f54-0', usage_metadata={'input_tokens': 90, 'output_tokens': 29, 'total_tokens': 119})

In [10]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it!\n"

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you! \n\nIs there anything I can help you with today?\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  \n\nI remember you telling me earlier. 😊  Is there something else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hello Krish! 👋 \n\nIt's nice to meet you.  \n\nI'm ready to answer your questions to the best of my ability. What can I help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 31, 'total_tokens': 76, 'completion_time': 0.081818182, 'prompt_time': 0.002351985, 'queue_time': 0.247202855, 'total_time': 0.084170167}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--726a630b-7833-4c3a-acf7-e8938d7401cf-0', usage_metadata={'input_tokens': 31, 'output_tokens': 45, 'total_tokens': 76})

In [15]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hello Krish, it's nice to meet you! \n\nI'm happy to help answer any questions you have to the best of my ability.  \n\nWhat can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 31, 'total_tokens': 78, 'completion_time': 0.085454545, 'prompt_time': 0.006749645, 'queue_time': 0.6203262540000001, 'total_time': 0.09220419}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a5a50193-5b9f-40e7-a612-767a1581a6ca-0', usage_metadata={'input_tokens': 31, 'output_tokens': 47, 'total_tokens': 78})

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish.  I remember! 😊  \n\nIs there anything else I can help you with?\n'

In [18]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [19]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण!  \n\nमुझे बहुत खुशी हो कि आप मुझसे बात करना चाह रहे हैं! मैं आपकी मदद करने के लिए तैयार हूँ।  आप मुझसे क्या पूछना चाहेंगे? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [20]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [21]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्ण! 😊  आप कैसे हैं? मैं आपकी मदद करने के लिए तैयार हूँ।  आप क्या जानना चाहेंगे? \n'

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)


In [23]:
response.content

'आपका नाम कृष्ण है।  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [24]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [25]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

" As an AI, I don't have access to your personal information, including your ice cream preferences.\n\nWhat's your favorite flavor?  🍦😊 \n\n"

In [26]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2". 😊'

In [27]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content